In [1]:
import cv2
import numpy as np

# Function to rescale each frame from the video

In [2]:
def rescale_frame(frame,scale):
    width = int(frame.shape[1] * scale) #getting current width of the frame and scaling it to the value provided
    height = int(frame.shape[0] * scale) #getting current height of the frame scaling it to the value provided
    dimensions = (width-100, height) #reducing 100units form the current width and creating a dimensions tuple
    return cv2.resize(frame, dimensions, interpolation=cv2.INTER_AREA) #passing the current frame, new dimensions tuple in resize function

# Loading the 'Haarcascade' file which will be used as the face detection algorithm

In [3]:
face_cascade=cv2.CascadeClassifier("cascades/haarcascade_frontalface_default.xml")

# Loading the model used for prediction

Changes were made to the network, tested on the test dataset and confusion matrix were generated for each model's predictions for the test data. The model with the most balanced confusin matrix was chosen to be the model of choice

In [4]:
from tensorflow.keras.models import load_model
model = load_model('model/face_mask_detection_model.h5')

# A function to to predict for each frame and convert the neumeric results to its respective class name, confidence etc

In [5]:
def predictions(image):

    frame_exp = np.expand_dims(image,axis=0) #increasing the image dimension to fit the dimension used while training
    pred_ = model.predict(x=frame_exp) #preicting the class
    norm_pred = np.argmax(pred_) #recieving the index of the class with the highest confidence value
    conf ="{:.2f}".format(np.max(pred_)*100) #formatting the confidence value
    
    #if prediction is 0 the classname will be 'Mask'
    if norm_pred==0:
        pred = 'Mask'
        addX = 230 #values for defining the rectangle around the classname text
        color=(0,255,0) #color of the rectangle

    #if prediction is 1 the classname will be 'Non Mask'
    else:
        pred = 'Non Mask'
        addX = 300
        color=(0,0,255)
    
    return pred,conf,color,addX        

# Capturing video from the webcam and performing the detection and prediction

In [6]:
IMG_SIZE = 227
video=cv2.VideoCapture(0)
address = 'http://192.168.0.4:8080//video'
video.open(address)

count = 0

while True:
    try:
        check,frame=video.read() #reading the video from the webcam
        frame_resized = cv2.resize(frame,(IMG_SIZE,IMG_SIZE)) #resizing the frame to the image size used in the model (227,227)
        frame_rescaled = rescale_frame(frame,0.5) #rescaling the frame using the function defined
        gray_frame = cv2.cvtColor(frame_rescaled,cv2.COLOR_BGR2GRAY) #converting the frame to gray for haarcascase face detection
        faces = face_cascade.detectMultiScale(gray_frame,scaleFactor=1.5,minNeighbors=5) #passung the gray_frame to haarcascade's detection method

        for x,y,w,h in faces: #interating over each frame and recieving the 4 points of the bounded region where the face is
            roi_gray = gray_frame[y:y+h,x:x+w] #defining the 4 points where the face exists for gray_frame
            roi_color = frame_rescaled[y:y+h,x:x+w] #defining the 4 points where the face exists for color_frame
            
            stroke = 2 #defining the stroke size for drawing the rectangle on the detected face
            end_cord_x = x+w #end coordinate of the width of the rectangle
            end_cord_y = y+h #end coordinate for the height of the rectangle
            
            face_only = cv2.resize(roi_color,(IMG_SIZE,IMG_SIZE)) #grabbing only the detected face, resizing it to (227,227)
            pred,conf,color,addX = predictions(face_only) #passing the face part to prediction function to get the predicted class
            
            cv2.rectangle(frame_rescaled,(x,y),(end_cord_x,end_cord_y),color,stroke) #drawing the rectangle using the start and end coordinates on the live video
            
            font = cv2.FONT_HERSHEY_SIMPLEX #defining the font style
            text = pred+':'+conf+'%' #defining the text to be shown
            stroke = 2 #defining the stroke size
            
            cv2.rectangle(frame_rescaled,(x,y-40),(x+addX,y),color,cv2.FILLED) #drawing a rectange arountd the text
            cv2.putText(frame_rescaled,text,(x,y-10),font,1,(255,255,255),stroke) #writing the prediction and confidence on the frame

        cv2.imshow('webcam',frame_rescaled) #show the webcam video
        key=cv2.waitKey(1)

        if key==ord('w'):
            break   
    except Exception as e: 
        print(e)
        break
        
video.release()
cv2.destroyWindow('webcam')